### **Chapter 04.05 학습 알고리즘 구현하기**

***신경망 학습 절차***

0. **전제**
  * 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 ***학습***이라고 한다.
1.   **미니배치**
  * 훈련 데이터 중 일부를 무작위로 가져오기 ➜ 미니배치
   * 미니배치의 손실 함수 값 줄이는 것이 목표
2.   **기울기 산출**
 * 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기 구함
 * 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시
3.   **매개변수 갱신**
 * 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
4.   **반복**
   * 1~3단계 반복


***확률적 경사 하강법 (SGD : Stochastic Gradient Descent)***

* 확률적으로 무작위로 골라낸 데이터 (미니배치)에 대해 수행하는 경사 하강법

<img width="35%" src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcGGhcs%2FbtqYAsMiu5M%2FuzRZ22SMGALicF6VveFjjk%2Fimg.png">

<img width="50%" src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FUS7rW%2FbtqYDj9a5Qc%2F78VFb2QlmeVUwkp0N1RH1k%2Fimg.png">

#### **4.5.1 2층 신경망 클래스 구현 (은닉층이 1개인 네트워크)**

In [ ]:
import numpy as np
from functions import *
from gradient import numerical_gradient

class TwoLayerNet:
  # 가중치 초기화 : 입력층 뉴런 수, 은닉층 뉴런 수, 출력층 뉴런 수
  def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
    # 신경망의 가중치 매개변수를 보관하는 딕셔너리 변수
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

  # 예측(추론) 실행
  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y

  # 손실 함수 값 계산 (x : 입력 데이터, t : 정답 레이블)
  def loss(self, x, t):
    y = self.predict(x)
    return cross_entropy_error(y, t)

  # 정확도 계산
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis = 1)
    t = np.argmax(t, axis = 1)

    accuracy = np.sum(y == t) / float(x.shape[0])

  # 수치 미분 방식으로 각 매개변수 기울기 계산 (x : 입력 데이터, t : 정답 레이블)
  def numerical_gradient(self, x, t):
    loss_W = lambda W : self.loss(x, t)

   # 기울기를 보관하는 딕셔너리 변수
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

  # 오차역전파법 사용하여 기울기 계산
  def gradient(self, x, t):
    return 0

#### **4.5.2 미니배치 학습 구현**

In [ ]:
from keras.datasets import mnist

(x_train, t_train), (x_test, t_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28**2)

train_loss_list = []

## 하이퍼파라미터
# 반복 횟수
iters_num = 10000
train_size = x_train.shape[0]
# 미니배치 크기
batch_size = 100
# 학습률
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

#### **4.5.3 시험 데이터로 평가**

***에폭 (epoch)***

* 1epoch 이란 전체 훈련 데이터셋에 대해 forward pass와 backward pass 과정을 모두 포함하여 전체적으로 한 번 학습했을 때를 말함
  * 예) 전체 훈련 데이터셋에 대해 50번 학습을 하면 epochs = 50
  * 예) 훈련 데이터 수 = 1000, epochs = 10, batch_size = 200
       * iteraction (데이터를 나누는 회수) = 1000 / 200 = 5
       * 즉, 1000개의 데이터를 200개로 나누어 5번 학습하는 것을 총 10번 (10 epochs) 반복

In [ ]:
from keras.datasets import mnist

(x_train, t_train), (x_test, t_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28**2)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

## 하이퍼파라미터
# 반복 횟수
iters_num = 10000
train_size = x_train.shape[0]
# 미니배치 크기
batch_size = 100
# 학습률
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  # 1에폭당 정확도 계산
  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)

    print("train acc, test acc : " + str(train_acc) + ", " + str(test_acc))